## Import modules

In [1]:
import streamlit as st
from streamlit_gsheets import GSheetsConnection
import streamlit_gsheets as stg
import pickle
import pandas as pd
import numpy as np

## Load data from gsheet

In [2]:
# Connections from google sheets


# conn = st.connection("gsheets", type=GSheetsConnection)
# existing_dhp = conn.read(worksheet="Data Histori Prediksi Suatu Prodi", ttl=5)
# existing_djm = conn.read(worksheet="Data Jumlah Mahasiswa")
# existing_formula = conn.read(worksheet="Rumus Pemantauan", usecols=list(range(7)), ttl=5)

# with open('existing_dhp.pickle', 'wb') as handle:
#     pickle.dump(existing_dhp, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('existing_djm.pickle', 'wb') as handle:
#     pickle.dump(existing_djm, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('existing_formula.pickle', 'wb') as handle:
#     pickle.dump(existing_formula, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Load data from pickle files

In [3]:
with open('existing_dhp.pickle', 'rb') as handle:
    existing_dhp = pickle.load(handle)

with open('existing_djm.pickle', 'rb') as handle:
    existing_djm = pickle.load(handle)

with open('existing_formula.pickle', 'rb') as handle:
    existing_formula = pickle.load(handle)
    
model = pickle.load(open(r"next_year_students_prediction.sav", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'existing_dhp.pickle'

In [4]:
'''
139:131
124
117
'''

model.predict([[124]])

array([117.94833193])

## Data preprocessing

In [5]:
# Djm = data jumlah mahasiswa
# Dhp = data history prediksi

existing_djm = existing_djm.dropna(how="all")
existing_djm = existing_djm.replace('#N/A ()', 0)
# existing_dhp = existing_dhp.dropna(how="all")
existing_formula = existing_formula.dropna(how="all")
existing_djm.columns = [str(i) for i in existing_djm.columns]

# Dropdown options for Lembaga
lembaga_options = existing_djm['Lembaga'].unique()
formula_options = existing_formula['Nama Rumus'].unique()

unused_column = ['Kode Prodi', 'Kode Prodi UGM', 'Kode Fakultas', 'Program Studi', 'BAN PT', 'Departemen', 'Kluster', 'PDDIKTI x BAN']
existing_djm = existing_djm.drop(unused_column, axis=1)
existing_djm.head()

/tmp/ipykernel_5615/2254467019.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  existing_djm = existing_djm.replace('#N/A ()', 0)


,Fakultas,Prodi,Jenjang,Lembaga,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,KKMK,Doktor Ilmu Kedokteran dan Kesehatan,S-3,BAN-PT,53,38,37,59,44,49,57,61,62,79,64
1,KKMK,Magister Ilmu Biomedik,S-2,LAM-PTKes,49,51,44,57,44,34,32,46,63,61,62
2,KKMK,Magister Ilmu Kedokteran Klinis,S-2,LAM-PTKes,75,96,82,51,39,48,80,101,50,147,139
3,KKMK,Magister Ilmu Kedokteran Tropis,S-2,LAM-PTKes,14,17,11,20,15,10,8,13,15,12,9
4,KKMK,Magister Ilmu Pendidikan Kedokteran dan Kesehatan,S-2,LAM-PTKes,5,13,12,12,5,10,9,16,16,18,13


In [6]:
formula_options

array(['Rumus Pertama BAN 2024', 'Rumus Pertama Infokom',
       'Rumus Pertama Tek', 'Rumus Pertama Kes', 'Rumus Pertama EMBA',
       'Rumus Pertama SAMA', 'Rumus Kedua BAN', 'Rumus Kedua Infokom',
       'Rumus Kedua Tek', 'Rumus Kedua Kes', 'Rumus Kedua Emba',
       'Rumus Kedua SAMA', 'update bugfix BAN', 'cek persen SAMA',
       'cek fom baru BAN', 'Tambah Banyak Data TS INFOKOM',
       'cek min jumlah BAN', 'rumus baru ban pp', 'rumus baru kes pp',
       'rumus baru tek pp', 'rumus baru sama pp', 'rumus baru infokom pp',
       'rumus baru emba pp', 'rumus baru emba jm',
       'rumus baru infokom jm', 'rumus baru sama jm',
       'rumus baru teknik jm', 'rumus baru pt kes jm',
       'rumus baru ban pt jm', 'coba jm sama', 'coba jm infokom'],
      dtype=object)

In [7]:
djm_prodi = existing_djm["Prodi"]

max_year = int(existing_djm.columns[-1])
min_year = int(existing_djm.columns[12])

formula_type_input = ["Sudah Ada", "Baru"]

input_predict_year = 2024
input_years_to_predict = 3
input_formula = formula_type_input[0]

input_last_year = input_predict_year - 1
print(min_year, max_year)

2021 2023


In [8]:
# Prepare output columns


existing_djm['Hasil Prediksi Pemantauan'] = [None]*len(existing_djm)

for i in range(input_years_to_predict):
    existing_djm[str(input_predict_year+i)] = [None]*len(existing_djm)

existing_djm['Hasil Proyeksi Prediksi Pemantauan'] = [None]*len(existing_djm)



existing_djm.head()

,Fakultas,Prodi,Jenjang,Lembaga,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,Hasil Prediksi Pemantauan,2024,2025,2026,Hasil Proyeksi Prediksi Pemantauan
0,KKMK,Doktor Ilmu Kedokteran dan Kesehatan,S-3,BAN-PT,53,38,37,59,44,49,57,61,62,79,64,None,None,None,None,None
1,KKMK,Magister Ilmu Biomedik,S-2,LAM-PTKes,49,51,44,57,44,34,32,46,63,61,62,None,None,None,None,None
2,KKMK,Magister Ilmu Kedokteran Klinis,S-2,LAM-PTKes,75,96,82,51,39,48,80,101,50,147,139,None,None,None,None,None
3,KKMK,Magister Ilmu Kedokteran Tropis,S-2,LAM-PTKes,14,17,11,20,15,10,8,13,15,12,9,None,None,None,None,None
4,KKMK,Magister Ilmu Pendidikan Kedokteran dan Kesehatan,S-2,LAM-PTKes,5,13,12,12,5,10,9,16,16,18,13,None,None,None,None,None


## Calculate existing formula

In [9]:
formula_options

array(['Rumus Pertama BAN 2024', 'Rumus Pertama Infokom',
       'Rumus Pertama Tek', 'Rumus Pertama Kes', 'Rumus Pertama EMBA',
       'Rumus Pertama SAMA', 'Rumus Kedua BAN', 'Rumus Kedua Infokom',
       'Rumus Kedua Tek', 'Rumus Kedua Kes', 'Rumus Kedua Emba',
       'Rumus Kedua SAMA', 'update bugfix BAN', 'cek persen SAMA',
       'cek fom baru BAN', 'Tambah Banyak Data TS INFOKOM',
       'cek min jumlah BAN', 'rumus baru ban pp', 'rumus baru kes pp',
       'rumus baru tek pp', 'rumus baru sama pp', 'rumus baru infokom pp',
       'rumus baru emba pp', 'rumus baru emba jm',
       'rumus baru infokom jm', 'rumus baru sama jm',
       'rumus baru teknik jm', 'rumus baru pt kes jm',
       'rumus baru ban pt jm', 'coba jm sama', 'coba jm infokom'],
      dtype=object)

In [10]:
selected_formulas_orgs = {}

for index, lembaga_name in enumerate(lembaga_options):
    # Filter formulas by the selected Lembaga
    formula_options = existing_formula[existing_formula['Lembaga'] == lembaga_name]['Nama Rumus'].unique()

    # Dropdown to select formula for the current Lembaga
    the_options = "\n".join([f"{ix[0]},{ix[1]}" for ix in enumerate(formula_options)])
    selected_formulas_orgs[lembaga_name] = formula_options[
        int(
                1#input(f'Pilih ummus untuk lembaga {lembaga_name} (angkanya aja)\n {the_options}')
            )
        ]
    
    # Only for streamlit output
    selected_formulas_name = existing_formula[(existing_formula['Nama Rumus'] == selected_formulas_orgs[lembaga_name]) & (existing_formula['Lembaga'] == lembaga_name)].iloc[0]

print('selected formulas orgs:', selected_formulas_orgs)

selected formulas orgs: {'BAN-PT': 'Rumus Kedua BAN', 'LAM-PTKes': 'Rumus Kedua Kes', 'LAM TEKNIK': 'Rumus Kedua Tek', 'LAMSAMA': 'Rumus Kedua SAMA', 'LAM INFOKOM': 'Rumus Kedua Infokom', 'LAMEMBA': 'Rumus Kedua Emba'}


In [11]:
data_predict_years = [f"{next_year}" for next_year in range(input_predict_year+1, input_predict_year+input_years_to_predict)]
data_predict_target= [f"{input_predict_year}"]

# Fungsi untuk menghitung persentase penurunan
def hitung_persentase_penurunan(data, predict_year):
    # data_mahasiswa_start_year = input_fields["input_jumlah_mahasiswa_ts0"]
    # end_year = predict_year
    # penurunan_total = (data[f"{end_year} (Prediksi)"] - data_mahasiswa_start_year)
    # persentase_penurunan = (penurunan_total / 2) * 100
    # return persentase_penurunan
    ts_1 = data[f"{predict_year}"]
    ts_0 = data[str(input_last_year)]
    penurunan = (ts_0 - ts_1) / ts_1

    persentase_penurunan = penurunan * 100

    return persentase_penurunan

# print('input_fields:', input_fields)
# input_fields
# Fungsi untuk menghitung persentase penurunan dengan lebih dari satu tahun data
def hitung_persentase_penurunan_lebih_dari_satu(index, data, predict_year, banyak_data_ts):
    print('function ppls: ', (index, 'existing_djm', predict_year, banyak_data_ts))
    global input_fields

    total_penurunan = 0
    
    # Iterasi dari TS-1 hingga TS-n (n = banyak_data_ts)
    for i in range(int(banyak_data_ts) - 1):
        if i == 0:
            data = data.iloc[index]
            ts_0 = data[str(predict_year)]
            ts_1 = data[str(input_last_year)]
            
        else:
            # input_fields = input_fields[]
            ts_1 = input_fields[f"input_jumlah_mahasiswa_ts{i}"][index]
            ts_0 = input_fields[f"input_jumlah_mahasiswa_ts{i-1}"][index]

        print(i, (ts_0), (ts_1))
        penurunan = (ts_0 - ts_1) / ts_1
        total_penurunan += penurunan
        
    
    # print('data:', type(data))
    
    # Hitung rata-rata penurunan
    
    rata_rata_penurunan = total_penurunan / (banyak_data_ts - 1)
    persentase_penurunan = rata_rata_penurunan * 100

    print(f'out {index}:', ' | ', total_penurunan, ' | ', persentase_penurunan)
    
    return round(-persentase_penurunan, 2)

# print(existing_djm.loc[5])
# hitung_persentase_penurunan_lebih_dari_satu(5, existing_djm, 2024, 5.0)

In [12]:
existing_djm[[str(2024)]].isna()
existing_djm.iloc[143]

Fakultas                                              Ilmu Budaya
Prodi                                 Magister Pengkajian Amerika
Jenjang                                                       S-2
Lembaga                                                    BAN-PT
2013                                                           24
2014                                                           18
2015                                                            9
2016                                                           16
2017                                                           19
2018                                                           19
2019                                                           25
2020                                                           12
2021                                                           14
2022                                                           19
2023                                                            7
Hasil Pred

In [13]:
model_results = {
    "index": [],
    "X": [],
    "Y": [],
}


In [32]:
existing_djm['index'] = existing_djm.index

for index, row in existing_djm.iterrows():
    lembaga_prodi = row['Lembaga']
    prodi_name = row['Prodi']        
    current_students = row[str(input_last_year)]
    tahun_tidak_lolos = f"Lebih dari {input_years_to_predict} Tahun ke Depan"  # Default value
    
    selected_formula = existing_formula[(existing_formula['Nama Rumus'] == selected_formulas_orgs[lembaga_prodi]) & (existing_formula['Lembaga'] == lembaga_prodi)].iloc[0]
    input_kriteria = selected_formula["Kriteria"]
    existing_djm.at[index, 'Kriteria Input'] = input_kriteria
    input_ambang_batas_jumlah = selected_formula["Ambang Batas (Jumlah)"]
        # looping isi prediksi
    tahun_tidak_lolos_found = False
    existing_djm.at[index, 'Hasil Proyeksi Prediksi Pemantauan'] = f"Lebih dari {input_years_to_predict} Tahun ke Depan"
    for i in range(input_years_to_predict):
        next_year = input_last_year + i
        
        column_name = next_year#f'{next_year} (Prediksi)'

        # Lakukan prediksi menggunakan model untuk current_students dari baris ini
        
        # print('X:', index, next_year, existing_djm.at[index, next_year])
        prediksi_mahasiswa = model.predict([[existing_djm.at[index, str(next_year)]]])[0]
        model_results['index'].append(index)
        model_results['X'].append(existing_djm.at[index, str(next_year)])
        model_results['Y'].append(prediksi_mahasiswa)
        
            
        existing_djm.at[index, str(column_name+1)] = round(prediksi_mahasiswa)
        input_ambang_batas_jumlah = selected_formula["Ambang Batas (Jumlah)"]
        
        # print('Y:', column_name+1, existing_djm.at[index, column_name+1])

        # Perbarui current_students untuk iterasi berikutnya
        # current_students = prediksi_mahasiswa

        # Cek apakah jumlah mahasiswa di bawah ambang batas untuk kriteria Jumlah Mahasiswa
        
        
        
        
        if (not tahun_tidak_lolos_found) and (input_kriteria=='Jumlah Mahasiswa') and (prediksi_mahasiswa<input_ambang_batas_jumlah):
            existing_djm.at[index, 'Hasil Proyeksi Prediksi Pemantauan'] = next_year+1
            tahun_tidak_lolos_found = True
            
        print('loop:', prediksi_mahasiswa, input_ambang_batas_jumlah, tahun_tidak_lolos_found, input_kriteria, existing_djm.at[index, 'Hasil Proyeksi Prediksi Pemantauan'])    

            # Cek apakah jumlah mahasiswa di bawah ambang batas untuk kriteria Persentase Penurunan
                
                
            

    
    if input_kriteria == "Persentase Penurunan":
        print(index, 'PERSENTASE PENURUNAN')
        input_ambang_batas_persen = existing_formula[(existing_formula['Nama Rumus'] == selected_formulas_orgs[existing_djm.at[index, 'Lembaga']]) & (existing_formula['Lembaga'] == existing_djm.at[index, 'Lembaga'])].iloc[0]['Ambang Batas (%)']
        input_ambang_batas_persen = 0 if np.isnan(input_ambang_batas_persen) else input_ambang_batas_persen
        input_banyak_data_ts = selected_formula["Banyak Data TS"]
        input_ambang_batas_jumlah = None
        input_fields = {}
        selected_formula = existing_formula[(existing_formula['Nama Rumus'] == selected_formulas_orgs[lembaga_prodi]) & (existing_formula['Lembaga'] == lembaga_prodi)].iloc[0]
        
        
        for i in range(int(input_banyak_data_ts)-1):
            field_name = f"input_jumlah_mahasiswa_ts{i}"            
            try:
                input_fields[field_name] = existing_djm[str(input_predict_year-i-1)]
                # print(f'jmlh {i} | {field_name} | {input_predict_year-i} | { existing_djm[input_predict_year-i]}')
            except KeyError:
                # MUNGKIN datanya ga cukup, misal pilih 2014, tapi datanya cmn ada dari 2013, trus ambil -3 tahun
                raise ValueError("TAHUNNYA KURANG BRO")    
    

        # dari looping function
        input_ambang_batas_persen = existing_formula[(existing_formula['Nama Rumus'] == selected_formulas_orgs[existing_djm.at[index, 'Lembaga']]) & (existing_formula['Lembaga'] == existing_djm.at[index, 'Lembaga'])].iloc[0]['Ambang Batas (%)']
        input_ambang_batas_persen = 0 if np.isnan(input_ambang_batas_persen) else input_ambang_batas_persen
        
        if input_banyak_data_ts > 2:
            persentase_penurunan = hitung_persentase_penurunan_lebih_dari_satu(index, existing_djm, input_predict_year, input_banyak_data_ts)
        else:
            persentase_penurunan = hitung_persentase_penurunan(existing_djm, input_predict_year)
            
        existing_djm.at[index, 'Hitung Persentase Penurunan'] = persentase_penurunan
        #print('persentase penurunan:', persentase_penurunan)
        # hasil_prediksi_pemantauan = "Lolos" if persentase_penurunan.values[0] <= input_ambang_batas_persen else "Tidak Lolos"
        hasil_prediksi_pemantauan = str(("Lolos" if persentase_penurunan <= input_ambang_batas_persen else "Tidak Lolos"))
        #"Lolos" if float(persentase_penurunan.iloc[1]) <= input_ambang_batas_persen else "Tidak Lolos"
        convert_percent_to_ambang_batas_jumlah_mahasiswa = int(row[str(input_last_year)] * (1 - input_ambang_batas_persen / 100))
        # ambang_batas_jumlah_mahasiswa = int(data_prodi["current_students"] * (1 - input_ambang_batas_persen / 100))
        existing_djm.at[index, "Persentase Penurunan Maksimal"] = f"{input_ambang_batas_persen}%"
        # existing_djm.at[index, "Ambang Batas Jumlah Mahasiswa Minimal"] = convert_percent_to_ambang_batas_jumlah_mahasiswa
        existing_djm.at[index, f"Hasil Prediksi Pemantauan ({input_predict_year})"] = hasil_prediksi_pemantauan
        existing_djm.at[index, "Persentase Penurunan Maksimal"]
        # print(f'row existing djm at index {index}: ', existing_djm.loc[index])
        for col, value in input_fields.items():
            if col!="input_jumlah_mahasiswa_ts0":
                existing_djm[col] = value
        ts = [f"input_jumlah_mahasiswa_ts{i}" for i in range(1, int(input_banyak_data_ts-1))]
        ts = sorted(ts, reverse=True)
     
    if input_kriteria == "Jumlah Mahasiswa":
        print(index, 'JUMLAH MAHASISWA')
        
        # input_jumlah_mahasiswa_ts = None
        # print('comparasion prediksi pemantauan:', existing_djm.at[index, str(input_predict_year)], input_ambang_batas_jumlah, input_predict_year)

        hasil_prediksi_pemantauan = str(("Lolos" if existing_djm.at[index, str(input_predict_year)] >= input_ambang_batas_jumlah else "Tidak Lolos"))
        input_ambang_batas_persen = None
        input_fields = None
        existing_djm.at[index, "Ambang Batas Jumlah Mahasiswa Minimal"] = input_ambang_batas_jumlah
        existing_djm.at[index, "Hasil Prediksi Pemantauan"] = hasil_prediksi_pemantauan



existing_djm[existing_djm.columns[7:]]

loop: 62.18233616352394 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
loop: 60.32346963809087 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
loop: 58.46460311265781 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
0 JUMLAH MAHASISWA
loop: 60.32346963809087 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
loop: 58.46460311265781 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
loop: 56.605736587224754 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
1 JUMLAH MAHASISWA
loop: 131.8898308672638 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
loop: 125.3837980282481 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
loop: 118.87776518923236 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
2 JUMLAH MAHASISWA
loop: 11.063506714114695 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
loop: 12.922373239547758 10.0 False Jumlah Mahasiswa Lebih dari 3 Tahun ke Depan
loop: 14.781239764980821 10.0 False Jumlah Mahasiswa Lebih 

,2016,2017,2018,2019,2020,2021,2022,2023,Hasil Prediksi Pemantauan,2024,2025,2026,Hasil Proyeksi Prediksi Pemantauan,index,Kriteria Input,Ambang Batas Jumlah Mahasiswa Minimal
0,59,44,49,57,61,62,79,64,Lolos,62,60,58,Lebih dari 3 Tahun ke Depan,0,Jumlah Mahasiswa,10.0
1,57,44,34,32,46,63,61,62,Lolos,60,58,57,Lebih dari 3 Tahun ke Depan,1,Jumlah Mahasiswa,10.0
2,51,39,48,80,101,50,147,139,Lolos,132,125,119,Lebih dari 3 Tahun ke Depan,2,Jumlah Mahasiswa,10.0
3,20,15,10,8,13,15,12,9,Lolos,11,13,15,Lebih dari 3 Tahun ke Depan,3,Jumlah Mahasiswa,10.0
4,12,5,10,9,16,16,18,13,Lolos,15,17,18,Lebih dari 3 Tahun ke Depan,4,Jumlah Mahasiswa,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,16,19,19,25,12,14,19,7,Tidak Lolos,9,11,13,2024,143,Jumlah Mahasiswa,10.0
144,0,0,0,0,18,15,13,22,Lolos,23,24,25,Lebih dari 3 Tahun ke Depan,144,Jumlah Mahasiswa,10.0
145,42,45,34,37,30,22,28,41,Lolos,41,41,41,Lebih dari 3 Tahun ke Depan,145,Jumlah Mahasiswa,10.0
146,41,48,52,33,36,30,25,25,Lolos,26,27,28,Lebih dari 3 Tahun ke Depan,146,Jumlah Mahasiswa,10.0


In [15]:
existing_djm[existing_djm['2024']==round(60.32346963809087)]

,Fakultas,Prodi,Jenjang,Lembaga,2013,2014,2015,2016,2017,2018,...,2022,2023,Hasil Prediksi Pemantauan,2024,2025,2026,Hasil Proyeksi Prediksi Pemantauan,index,Kriteria Input,Ambang Batas Jumlah Mahasiswa Minimal
1,KKMK,Magister Ilmu Biomedik,S-2,LAM-PTKes,49,51,44,57,44,34,...,61,62,Lolos,60,58,57,Lebih dari 3 Tahun ke Depan,1,Jumlah Mahasiswa,10.0
98,ISIPOL,Magister Ilmu Hubungan Internasional,S-2,BAN-PT,89,95,40,39,27,27,...,41,62,Lolos,60,58,57,Lebih dari 3 Tahun ke Depan,98,Jumlah Mahasiswa,10.0


In [16]:
existing_djm[]

SyntaxError: invalid syntax (3354081866.py, line 1)

In [ ]:
print(existing_djm.columns)

Index(['Fakultas', 'Prodi', 'Jenjang', 'Lembaga', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
       'Hasil Prediksi Pemantauan', '2024', '2025', '2026',
       'Hasil Proyeksi Prediksi Pemantauan', 'Kriteria Input'],
      dtype='object')


In [ ]:
- 'Hitung Persentase Penurunan',
- 'Persentase Penurunan Maksimal'
-'input_jumlah_mahasiswa_ts
+ ambang batas mahsiswa jumlah minimal
+ hasil proyeksi prediksi pemantauan

SyntaxError: unterminated string literal (detected at line 3) (1239342349.py, line 3)